# Data Set Generation

In [ ]:
import datetime
import pandas as pd
import numpy as np
#from sktime.forecasting.model_selection import temporal_train_test_split
from sklearn.model_selection import train_test_split, GroupShuffleSplit
from matplotlib import pyplot as plt
import janitor
from sklearn.linear_model import LinearRegression

SEED = 0
np.random.seed(SEED)
pd.set_option('display.max_columns', None)

In [ ]:
longitudinal_ehr = pd.read_csv('./data/updated_raw_data_7_29_22.csv', index_col=0, parse_dates=['HIRE_DATE', 'TERM_DATE', 'ReportingPeriodStartDate'])
#longitudinal_ehr = pd.read_csv('./data/ZZ_data_zz/turbo_data_10-10-21_deID.csv', index_col=0, parse_dates=['HIRE_DATE', 'TERM_DATE', 'ReportingPeriodStartDate'])
#longitudinal_ehr_age = pd.read_csv('longitudinal_ehr_age_group.csv', index_col=0, parse_dates=['hire_date', 'term_date', 'reportingperiodstartdate'])

In [ ]:
longitudinal_ehr['physician_id'] = longitudinal_ehr.index
longitudinal_ehr = longitudinal_ehr.reset_index(drop=True)
longitudinal_ehr = longitudinal_ehr.clean_names(remove_special=True)
longitudinal_ehr = longitudinal_ehr.sort_values(['physician_id','reportingperiodstartdate'])
longitudinal_ehr


In [ ]:
# look at some basic stats for mal vs female
longitudinal_ehr[longitudinal_ehr['gender'] == 'Female']['specialty'].value_counts()

In [ ]:
longitudinal_ehr['reportingperiodstartdate'].max()

In [ ]:
longitudinal_ehr['reportingperiodstartdate'].min()

In [ ]:
(longitudinal_ehr['reportingperiodstartdate'].max()-longitudinal_ehr['reportingperiodstartdate'].min())

In [ ]:
longitudinal_ehr['date_yyyymm'].min()

In [ ]:
longitudinal_ehr['date_yyyymm'].max()

In [ ]:
assert all(longitudinal_ehr.groupby(['physician_id'])['reportingperiodstartdate'].is_monotonic_increasing), 'Not monotonic increasing'

In [ ]:
longitudinal_ehr.groupby(['physician_id'])['reportingperiodstartdate'].is_monotonic_increasing

In [ ]:
print(f"num physcicians: {len(longitudinal_ehr['physician_id'].unique())}")
print(f"num quit: {sum(~longitudinal_ehr.groupby('physician_id').last()['term_date'].isna())}")

In [ ]:
# Add in the time_8 entries
ehr_time_8 = (((longitudinal_ehr['time_in_system_per_day_num']/60)/(longitudinal_ehr['sched_time_appts_in_hrs']))*8)
wow_time_8 = (((longitudinal_ehr['time_outside_scheduled_hours_num'] + longitudinal_ehr['time_on_unscheduled_days_value'])/60)/longitudinal_ehr['sched_time_appts_in_hrs']) * 8
note_time_8 = ((longitudinal_ehr['time_in_notes_per_day_num']/60)/(longitudinal_ehr['sched_time_appts_in_hrs']))*8
order_time_8 = ((longitudinal_ehr['time_in_orders_per_day_num']/60)/(longitudinal_ehr['sched_time_appts_in_hrs']))*8
ib_time_8 = ((longitudinal_ehr['time_in_in_basket_per_day_num']/60)/(longitudinal_ehr['sched_time_appts_in_hrs']))*8
review_time_8 = ((longitudinal_ehr['time_in_clinical_review_per_day_num']/60)/(longitudinal_ehr['sched_time_appts_in_hrs']))*8
longitudinal_ehr['ehr_time_8'] = ehr_time_8
longitudinal_ehr['wow_time_8'] = wow_time_8
longitudinal_ehr['note_time_8'] = note_time_8
longitudinal_ehr['order_time_8'] = order_time_8
longitudinal_ehr['ib_time_8'] = ib_time_8
longitudinal_ehr['review_time_8'] = review_time_8

In [ ]:
# constants for analysis
n_months = 6
forecast_lead_time = n_months*30 # forecast departures within this lead time interval, days

In [ ]:
# Want to flag physicians on the months leading up to the quit NOT if they have quit within x days 
# of study date as they may have been hired at a different time
# find time to departure and observations with time to departure meeting criterion
longitudinal_ehr['time_to_departure'] = (longitudinal_ehr['term_date'] - longitudinal_ehr['reportingperiodstartdate']).dt.days
#plt.hist(data['time_to_departure'])
#remnove 0's their last n-1 observations (last 6 months?)
# if we remove the last 6 months we can guarentee that they didnt quit in the last 6 months. otherwise we cant assign that label
longitudinal_ehr['departure_in_interval'] = longitudinal_ehr['time_to_departure'] < forecast_lead_time

# hire date to tenure in approx years
longitudinal_ehr['tenure'] = (longitudinal_ehr['reportingperiodstartdate'] - longitudinal_ehr['hire_date']).dt.days/365

# re-code reporting period start
first_obs = min(longitudinal_ehr['reportingperiodstartdate'])
longitudinal_ehr['study_day'] = (longitudinal_ehr['reportingperiodstartdate'] - first_obs).dt.days

# variable for month of observation
longitudinal_ehr['calendar_month'] = longitudinal_ehr['reportingperiodstartdate'].dt.month

# Continuous age mapped to age range number
bins = [24, 34, 44, 54, 64, np.inf]
names = ['25-34', '35-44', '45-54', '55-64', '65+']
names = [0, 1, 2, 3, 4]
longitudinal_ehr['age_group'] = pd.cut(longitudinal_ehr['age_as_of_06_30_2021'], bins, labels=names).tolist()
#longitudinal_ehr = longitudinal_ehr.drop('age_as_of_06_30_2021', axis=1)
# Convert reporting period start date into covid waves
from datetime import datetime, timedelta
bins = [
    longitudinal_ehr['reportingperiodstartdate'].min() - timedelta(days = 1),# february and before
    datetime.strptime('Mar 2020', '%b %Y'),
    datetime.strptime('Jul 2020', '%b %Y'),
    datetime.strptime('Nov 2020', '%b %Y'),
    datetime.strptime('Apr 2021', '%b %Y'),# april and after 
    longitudinal_ehr['reportingperiodstartdate'].max() + timedelta(days = 1),
]
names = [0, 1, 2, 3, 4]
longitudinal_ehr['covid_wave'] = pd.cut(longitudinal_ehr['reportingperiodstartdate'], bins, labels=names).tolist()

# gender to integer level
longitudinal_ehr['gender'] = pd.factorize(longitudinal_ehr['gender'])[0]

## drop rows that don't make sense
#keep_rows = (data['time_to_departure'] < 0) | (data['time_to_departure'].isna())
longitudinal_ehr = longitudinal_ehr[(longitudinal_ehr['time_to_departure'] >= 0) | (longitudinal_ehr['time_to_departure'].isna())].copy()

In [ ]:
def custom_round(x, base=1):
    if pd.isna(x):
        return x
    return int(base * round(float(x)/base))

In [ ]:
longitudinal_ehr['tenure']

In [ ]:
longitudinal_ehr['tenure'] = longitudinal_ehr['tenure'].apply(lambda x: custom_round(x, 5))

In [ ]:
longitudinal_ehr['tenure']

In [ ]:
print(f'num obs without quit: {longitudinal_ehr.time_to_departure.isna().sum()}')
print(f'num obs with quit: {longitudinal_ehr.departure_in_interval.sum()}')

In [ ]:
# how many physicians, quit and did not
print(f"num physicians: {len(longitudinal_ehr['physician_id'].unique())}")
print(f"num retained: {sum(longitudinal_ehr.groupby('physician_id')['term_date'].first().isna())}")
print(f"num departed: {sum(~longitudinal_ehr.groupby('physician_id')['term_date'].first().isna())}")

In [ ]:
# obs per physician
longitudinal_ehr.groupby('physician_id')['physician_id'].count().value_counts().sort_index(ascending = False).head(3)

In [ ]:
tenure = (longitudinal_ehr.groupby('physician_id')['reportingperiodstartdate'].first() - longitudinal_ehr.groupby('physician_id')['hire_date'].first())
tenure = np.round(tenure.dt.days/365)
tenure.value_counts().head(3) # negative tenures at start?

In [ ]:
# drop columns that aren't needed
cols_to_drop = [
                    #'physician_id',
                    #'reportingperiodstartdate',
                    #'tenure',
                    'study_day',
                    'reportingperiodenddate',
                    'term_date',
                    #'time_to_departure',
                    'hire_date',
                    #'age_range_as_of_12_31_19',
                    'date_yyyymm'
                ]
physician_id, unique_physician_ids = pd.factorize(longitudinal_ehr.physician_id)
longitudinal_ehr['physician_id'] = physician_id
longitudinal_ehr = longitudinal_ehr.drop(cols_to_drop, axis = 1)

In [ ]:
categorical_cols = ['provtype','physician_id', 'specialty', 'age_group', 'gender', 'departure_in_interval', 'calendar_month', 'covid_wave']
continuous_cols = [  'patient_volume',
                     'physician_demand',
                     'physician_work_intensity',
                     'panel_cnt',
                     'risk_avg',
                     'teamwork_on_inbox_value',
                     'note_quality_manual_value',
                     'note_quality_contribution_value',
                     'number_of_rx_errors',
                     'ehr_time_8',
                     'wow_time_8',
                     'note_time_8',
                     'order_time_8',
                     'ib_time_8',
                     'review_time_8',
                     'tenure',# 'study_day'
                      ]
date_cols = ['reportingperiodstartdate']#like the age or similar

In [ ]:
longitudinal_ehr = longitudinal_ehr[categorical_cols + continuous_cols + date_cols]

In [ ]:
sp_map = pd.read_csv('./references/specialty_dictionary.csv',index_col='old').to_dict()['new']
# sc = longitudinal_ehr.specialty.value_counts()
specialty_mapped = longitudinal_ehr.specialty.copy()
specialty_mapped2 = specialty_mapped.map(sp_map)
sp_map_comp = pd.DataFrame({'old': specialty_mapped,'new': specialty_mapped2})

In [ ]:
## one-hot encode categorical cols
encode_cols = ['provtype', 'specialty']
longitudinal_ehr = pd.get_dummies(longitudinal_ehr, columns = encode_cols)

In [ ]:
def ols_window(df, y_col, x_col = 'index'):
# compute least squares slope from y ~ x from RollingGroupby object as produced by .rolling()
# treats multiple y_col as 
# returns nan if < 2 data points
# by default treats index as the x variable
    slopes = np.full((1,len(y_col)),np.nan)
    if len(df) >= 2:

        # get the X
        if x_col == 'index':
            X = df.index.to_numpy()
            X = X.reshape(X.shape[0],-1) # needs to be 2d col vector
        else:
            X = df[[x_col]]              # needs to be 2d col vector
        
        
        
        for idx,cur_col in enumerate(y_col):
            y = df[cur_col]                    # ok as true 1d vector
            if not np.any(np.isnan(y)):
                lm = LinearRegression().fit(X, y)
                slope = lm.coef_[0]
                slopes[0,idx] = slope

    return(slopes)   
    #return(pd.DataFrame(slopes,columns=y_col))

def ols_rolling(df, window, group_col, y_col, x_col):
    
    # compute rolling slope on columns of pd data frame
    # this has not been optimized for speed or general use outside of our workflow

    if x_col != 'index':
        keep_cols = y_col+[x_col]
    else:
        keep_cols = y_col

    roll_iter = df.groupby(group_col)[keep_cols].rolling(window,on=x_col)

    df_rolling = pd.DataFrame(np.full((len(df),len(y_col)),np.nan),columns = y_col)

    for idx,x in enumerate(roll_iter):
        slope = ols_window(x, x_col = 'index', y_col = y_col)
        df_rolling.loc[idx] = slope
    
    return(df_rolling)



In [ ]:
# Make the rolling average columns for the time dependent covariates
time_dep_cov_cols = ['patient_volume',
                     'physician_demand',
                     'physician_work_intensity',
                     'panel_cnt',
                     'risk_avg',
                     'teamwork_on_inbox_value',
                     'note_quality_manual_value',
                     'note_quality_contribution_value',
                     'number_of_rx_errors',
                     'ehr_time_8',
                     'wow_time_8',
                     'note_time_8',
                     'order_time_8',
                     'ib_time_8',
                     'review_time_8']
n_steps = 2
def roll_diff(x):
    return x.iloc[-1] - x.iloc[0]
# exponential weighted mean
EWA_indicators = longitudinal_ehr.groupby('physician_id')[time_dep_cov_cols + ['reportingperiodstartdate']].ewm(halflife='64D',times='reportingperiodstartdate').mean()
EWA_indicators = EWA_indicators.add_prefix('EWA_avg_')

orig_index = longitudinal_ehr.index
orig_cols = set(longitudinal_ehr.columns)
# longitudinal_ehr = longitudinal_ehr.merge(rolling_indicators, how='inner', left_on= orig_index,right_on = rolling_indicators.index.get_level_values(1)).drop(['key_0', 'r_avg_reportingperiodstartdate'], axis=1).set_index(orig_index)
longitudinal_ehr = longitudinal_ehr.merge(EWA_indicators, how='inner', left_on= orig_index,right_on = EWA_indicators.index.get_level_values(1)).drop(['key_0',], axis=1).set_index(orig_index)
# longitudinal_ehr = longitudinal_ehr.merge(rolling_diff, how='inner', left_on= orig_index,right_on = rolling_diff.index.get_level_values(1)).drop(['key_0', 'rolling_diff_reportingperiodstartdate'], axis=1).set_index(orig_index)


In [ ]:
assert all(longitudinal_ehr.groupby(['physician_id'])['reportingperiodstartdate'].is_monotonic_increasing), 'Not monotonic increasing'

In [ ]:
rolling_slope = ols_rolling(
    longitudinal_ehr,
    group_col = 'physician_id',
    y_col = time_dep_cov_cols,
    x_col = 'reportingperiodstartdate',
    window='64D'
)
rolling_slope = rolling_slope.add_prefix('r_slope_')

In [ ]:
longitudinal_ehr = pd.concat([longitudinal_ehr,rolling_slope.set_index(longitudinal_ehr.index)],axis=1) # does same job as merge statement
continuous_cols += list(set(longitudinal_ehr.columns) - orig_cols)

In [ ]:
longitudinal_ehr.columns

In [ ]:
#pd.to_pickle(longitudinal_ehr,"./data/processed/turbo_7_18_22_deid_processed_3.pkl")

In [ ]:
#pd.to_pickle(longitudinal_ehr,"./data/processed/turbo_7_18_22_deid_processed_3_NO_TENURE_STUDYDAY.pkl")

In [ ]:
longitudinal_ehr.groupby('physician_id').last()['departure_in_interval'].value_counts()

In [ ]:
counter = 0
for x in longitudinal_ehr['physician_id'].unique():
    if sum(longitudinal_ehr[longitudinal_ehr['physician_id'] == x]['departure_in_interval']) > 0:
        counter += 1
counter

In [ ]:
longitudinal_ehr.columns

In [ ]:
pd.to_pickle(longitudinal_ehr,"./data/processed/turbo_7_29_22_deid_processed_3_ROUND_5y_TENURE_NO_STUDYDAY.pkl")

In [ ]:
sum(longitudinal_ehr['departure_in_interval'] == True)

In [ ]:
a = ['age_group', 'gender', 'calendar_month', 'covid_wave', 'patient_volume',
       'physician_demand', 'physician_work_intensity', 'panel_cnt', 'risk_avg',
       'teamwork_on_inbox_value', 'note_quality_manual_value',
       'note_quality_contribution_value', 'number_of_rx_errors', 'ehr_time_8',
       'wow_time_8', 'note_time_8', 'order_time_8', 'ib_time_8',
       'review_time_8', 'tenure', 'specialty_Allergy',
       'EWA_avg_patient_volume', 'EWA_avg_physician_demand',
       'EWA_avg_physician_work_intensity', 'EWA_avg_panel_cnt',
       'EWA_avg_risk_avg', 'EWA_avg_teamwork_on_inbox_value',
       'EWA_avg_note_quality_manual_value',
       'EWA_avg_note_quality_contribution_value',
       'EWA_avg_number_of_rx_errors', 'EWA_avg_ehr_time_8',
       'EWA_avg_wow_time_8', 'EWA_avg_note_time_8', 'EWA_avg_order_time_8',
       'EWA_avg_ib_time_8', 'EWA_avg_review_time_8', 'r_slope_patient_volume',
       'r_slope_physician_demand', 'r_slope_physician_work_intensity',
       'r_slope_panel_cnt', 'r_slope_risk_avg',
       'r_slope_teamwork_on_inbox_value', 'r_slope_note_quality_manual_value',
       'r_slope_note_quality_contribution_value',
       'r_slope_number_of_rx_errors', 'r_slope_ehr_time_8',
       'r_slope_wow_time_8', 'r_slope_note_time_8', 'r_slope_order_time_8',
       'r_slope_ib_time_8', 'r_slope_review_time_8']

In [ ]:
len(a)